In [6]:
# đặt biến môi trường 
%env SPARK_LOCAL_HOSTNAME=localhost

env: SPARK_LOCAL_HOSTNAME=localhost


In [7]:
#khởi động spark
import findspark
findspark.init()

In [12]:
# thêm các thư viện spark để thực hiện random 
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from pyspark.python.pyspark.shell import spark
import os

In [8]:
#Load dữ liệu lên
data = spark.read.load("C:/Users/Admin/Documents/Absenteeism_at_work.csv", format="csv", header=True, delimiter=";")

data = data.withColumn("MOA", data["Month of absence"] - 0).withColumn("label", data['Transportation expense'] - 0). \
    withColumn("ROA", data["Reason for absence"] - 0). \
    withColumn("distance", data["Distance from Residence to Work"] - 0). \
    withColumn("BMI", data["Body mass index"] - 0)

assem = VectorAssembler(inputCols=["label", "distance"], outputCol='features')

data = assem.transform(data)

labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# chia tập dữ liệu ra để test và train. 30 cho test 70 cho train
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Chuyển label
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

y_true = data.select("BMI").rdd.flatMap(lambda x: x).collect()
y_pred = data.select("ROA").rdd.flatMap(lambda x: x).collect()

pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.
model = pipeline.fit(trainingData)

# dự đoán
predictions = model.transform(testData)


predictions.select("predictedLabel", "label", "features").show(5)

evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

rfModel = model.stages[2]
print(rfModel)
print("Random Forest - Test Accuracy = %g" % (accuracy))
print("Random Forest - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Random Forest Model is :\n" + str(confusionmatrix))

print("The precision score for Random Forest Model is: " + str(precision))

print("The recall score for Random Forest Model is: " + str(recall))

+--------------+-----+------------+
|predictedLabel|label|    features|
+--------------+-----+------------+
|         235.0|235.0|[235.0,11.0]|
|         235.0|235.0|[235.0,11.0]|
|         235.0|235.0|[235.0,11.0]|
|         235.0|235.0|[235.0,11.0]|
|         235.0|235.0|[235.0,11.0]|
+--------------+-----+------------+
only showing top 5 rows

RandomForestClassificationModel: uid=RandomForestClassifier_d1a7fd57b44d, numTrees=10, numClasses=24, numFeatures=2
Random Forest - Test Accuracy = 0.908654
Random Forest - Test Error = 0.0913462
The Confusion Matrix for Random Forest Model is :
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]
The precision score for Random Forest Model is: 0.02972972972972973
The recall score for Random Forest Model is: 0.02972972972972973
